In [1]:
!pip install ibm-watsonx-ai

In [2]:
from ibm_watsonx_ai import APIClient, Credentials
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.foundation_models.prompts import PromptTemplateManager
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
import os

In [3]:
API_KEY = "MgGbYitfDU36Z8tw109g6J8udk9k3CKLCehTvXHblAua"
PROJECT_ID = "abc6e80c-882f-4805-80de-f0ce5b761420"
REGION = "us-south"
PROMPT_TEMPLATE_ASSET_ID = "55dafe85-8452-45bd-84cb-54005f25397d"
URL = f"https://{REGION}.ml.cloud.ibm.com"

In [4]:
credentials = Credentials(
    api_key=API_KEY,
    url=URL
)

client = APIClient(credentials)
client.set.default_project(PROJECT_ID)


'SUCCESS'

In [5]:
model_inference = ModelInference(
    model_id="ibm/granite-3-8b-instruct",
    credentials=credentials,
    project_id=PROJECT_ID
)

In [6]:
try:
    # Initialize PromptTemplateManager
    prompt_mgr = PromptTemplateManager(
        credentials=credentials,
        project_id=PROJECT_ID
    )
    
    # Define the prompt variables for the template
    prompt_variables = {
        "level": "beginner",           # Learner proficiency level
        "subject": "mathematics",      # Academic subject name
        "content": "quadratic equations and their solutions using the quadratic formula"  # Academic material to simplify
    }
    
    # Load the prompt template and fill variables
    from ibm_watsonx_ai.foundation_models.utils.enums import PromptTemplateFormats
    
    # Load the template as a string
    prompt_text = prompt_mgr.load_prompt(
        prompt_id=PROMPT_TEMPLATE_ASSET_ID,
        astype=PromptTemplateFormats.STRING
    )
    
    # Fill the template with variables
    filled_prompt = prompt_text.format(**prompt_variables)
    
    print(f"Using Prompt Template: {PROMPT_TEMPLATE_ASSET_ID}")
    print(f"Filled Prompt:\n{filled_prompt}\n")
    
    # Parameters for text generation
    generation_params = {
        GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
        GenParams.MAX_NEW_TOKENS: 200,
        GenParams.MIN_NEW_TOKENS: 50,
        GenParams.STOP_SEQUENCES: ["\n\n"]
    }
    
    # Generate response using the filled prompt
    response = model_inference.generate_text(
        prompt=filled_prompt,
        params=generation_params
    )
    
    print("Generated Response:")
    print(response)
    
except Exception as e:
    print(f"Error generating response: {str(e)}")
    print("Note: Make sure PROMPT_TEMPLATE_ASSET_ID is set to your actual template asset ID")

Using Prompt Template: 55dafe85-8452-45bd-84cb-54005f25397d
Filled Prompt:
You are a Course Content Simplification Agent. Your task is to analyze academic content and adapt it to different learning levels.

INPUTS:
- Original academic content: quadratic equations and their solutions using the quadratic formula
- Target learning level: beginner
- Subject area: mathematics

INSTRUCTIONS:
1. Analyze the complexity of the original content in mathematics
2. Identify jargon, complex concepts, and prerequisites
3. Rewrite the content for {level} learning level using:
   - Beginner: Simple language, analogies, step-by-step explanations, minimal jargon
   - Intermediate: Moderate technical terms with explanations, some complexity
   - Advanced: Full technical vocabulary, assume prior knowledge

4. Structure your response as:
   - **Complexity Analysis**: What makes this content difficult?
   - **Key Concepts Identified**: Main ideas to preserve
   - **Simplified Version**: Rewritten content for

In [7]:
try:
    # Initialize PromptTemplateManager
    prompt_mgr = PromptTemplateManager(
        credentials=credentials,
        project_id=PROJECT_ID
    )
    
    # List available prompt templates (optional - for reference)
    print("Available Prompt Templates:")
    templates = prompt_mgr.list()
    
    # Check the structure of the response and handle different formats
    if isinstance(templates, dict):
        if 'resources' in templates:
            template_list = templates['resources']
        elif 'results' in templates:
            template_list = templates['results']
        else:
            # If it's a different structure, just show the keys
            print(f"Response structure: {list(templates.keys())}")
            template_list = []
    elif isinstance(templates, list):
        template_list = templates
    else:
        print(f"Unexpected response type: {type(templates)}")
        template_list = []
    
    # Display templates
    if template_list:
        for i, template in enumerate(template_list[:5]):  # Show first 5
            if isinstance(template, dict):
                name = template.get('metadata', {}).get('name', 'Unknown')
                asset_id = template.get('metadata', {}).get('asset_id', template.get('id', 'Unknown'))
                print(f"- {name} (ID: {asset_id})")
            else:
                print(f"- Template {i}: {template}")
    else:
        print("No templates found or unable to parse response")
    
    # Load the specific prompt template
    from ibm_watsonx_ai.foundation_models.utils.enums import PromptTemplateFormats
    
    prompt_text = prompt_mgr.load_prompt(
        prompt_id=PROMPT_TEMPLATE_ASSET_ID,
        astype=PromptTemplateFormats.STRING
    )
    
    print(f"\nLoaded Prompt Template Text:\n{prompt_text}")
    
except Exception as e:
    print(f"Error with PromptTemplateManager: {str(e)}")
    print("This might be due to:")
    print("1. Invalid PROMPT_TEMPLATE_ASSET_ID")
    print("2. Insufficient permissions")
    print("3. Template not found in the specified project")

Available Prompt Templates:
Unexpected response type: <class 'pandas.core.frame.DataFrame'>
No templates found or unable to parse response

Loaded Prompt Template Text:
You are a Course Content Simplification Agent. Your task is to analyze academic content and adapt it to different learning levels.

INPUTS:
- Original academic content: {content}
- Target learning level: {level}
- Subject area: {subject}

INSTRUCTIONS:
1. Analyze the complexity of the original content in {subject}
2. Identify jargon, complex concepts, and prerequisites
3. Rewrite the content for {{level}} learning level using:
   - Beginner: Simple language, analogies, step-by-step explanations, minimal jargon
   - Intermediate: Moderate technical terms with explanations, some complexity
   - Advanced: Full technical vocabulary, assume prior knowledge

4. Structure your response as:
   - **Complexity Analysis**: What makes this content difficult?
   - **Key Concepts Identified**: Main ideas to preserve
   - **Simplified

In [8]:
try:
    # Define variables to fill in the template
    template_variables = {
        "level": "intermediate",
        "subject": "physics", 
        "content": "Newton's laws of motion and their applications in everyday life"
    }
    
    # Fill the prompt template with variables
    filled_prompt = prompt_text.format(**template_variables)
    
    print(f"Filled Prompt:\n{filled_prompt}\n")
    
    # Generate response using the filled prompt
    generation_params = {
        GenParams.DECODING_METHOD: DecodingMethods.SAMPLE,
        GenParams.TEMPERATURE: 0.7,
        GenParams.TOP_P: 0.9,
        GenParams.MAX_NEW_TOKENS: 250
    }
    
    response = model_inference.generate_text(
        prompt=filled_prompt,
        params=generation_params
    )
    
    print("Generated Response:")
    print(response)
    
except Exception as e:
    print(f"Error filling template or generating: {str(e)}")

Filled Prompt:
You are a Course Content Simplification Agent. Your task is to analyze academic content and adapt it to different learning levels.

INPUTS:
- Original academic content: Newton's laws of motion and their applications in everyday life
- Target learning level: intermediate
- Subject area: physics

INSTRUCTIONS:
1. Analyze the complexity of the original content in physics
2. Identify jargon, complex concepts, and prerequisites
3. Rewrite the content for {level} learning level using:
   - Beginner: Simple language, analogies, step-by-step explanations, minimal jargon
   - Intermediate: Moderate technical terms with explanations, some complexity
   - Advanced: Full technical vocabulary, assume prior knowledge

4. Structure your response as:
   - **Complexity Analysis**: What makes this content difficult?
   - **Key Concepts Identified**: Main ideas to preserve
   - **Simplified Version**: Rewritten content for intermediate level
   - **Learning Aids**: Suggest analogies, examp

In [9]:
try:
    # Note: For direct prompt template usage with PROMPT_VARIABLES, you need a deployed prompt template
    # This is the correct way to use PROMPT_VARIABLES parameter
    
    # If you have a deployed prompt template, uncomment and use this approach:
    # DEPLOYMENT_ID = "your_deployment_id_here"  # Set this to your deployed template ID
    
    # deployment_inference = ModelInference(
    #     deployment_id=DEPLOYMENT_ID,
    #     credentials=credentials,
    #     project_id=PROJECT_ID
    # )
    
    # deployment_params = {
    #     GenParams.PROMPT_VARIABLES: {
    #         "level": "advanced",
    #         "subject": "chemistry",
    #         "content": "chemical bonding and molecular structure"
    #     },
    #     GenParams.MAX_NEW_TOKENS: 200
    # }
    
    # response = deployment_inference.generate(params=deployment_params)
    # print("Deployment Response:")
    # print(response['results'][0]['generated_text'])
    
    print("Deployment method is commented out.")
    print("To use PROMPT_VARIABLES parameter, you need to:")
    print("1. Deploy your prompt template first")
    print("2. Use ModelInference with deployment_id instead of model_id")
    print("3. Then you can use the PROMPT_VARIABLES parameter in generate()")
    
except Exception as e:
    print(f"Error with deployment inference: {str(e)}")

Deployment method is commented out.
To use PROMPT_VARIABLES parameter, you need to:
1. Deploy your prompt template first
2. Use ModelInference with deployment_id instead of model_id
3. Then you can use the PROMPT_VARIABLES parameter in generate()


In [10]:
def simplify_content(level, subject, content):
    """
    Simplify academic content using the prompt template
    
    Args:
        level (str): Learning level (beginner, intermediate, advanced)
        subject (str): Academic subject
        content (str): Content to be simplified
        
    Returns:
        str: Simplified content or None if error
    """
    try:
        # Load the template
        prompt_mgr = PromptTemplateManager(
            credentials=credentials,
            project_id=PROJECT_ID
        )
        
        from ibm_watsonx_ai.foundation_models.utils.enums import PromptTemplateFormats
        prompt_text = prompt_mgr.load_prompt(
            prompt_id=PROMPT_TEMPLATE_ASSET_ID,
            astype=PromptTemplateFormats.STRING
        )
        
        # Fill variables
        variables = {
            "level": level,
            "subject": subject,
            "content": content
        }
        
        filled_prompt = prompt_text.format(**variables)
        
        # Generate response
        params = {
            GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
            GenParams.MAX_NEW_TOKENS: 300,
            GenParams.TEMPERATURE: 0.7
        }
        
        response = model_inference.generate_text(
            prompt=filled_prompt,
            params=params
        )
        
        return response
    
    except Exception as e:
        print(f"Error simplifying content: {str(e)}")
        return None


def batch_simplify_content(content_list):
    """
    Simplify multiple pieces of content
    
    Args:
        content_list (list): List of dictionaries with 'level', 'subject', 'content' keys
        
    Returns:
        list: List of simplified content results
    """
    results = []
    for item in content_list:
        result = simplify_content(
            level=item['level'],
            subject=item['subject'], 
            content=item['content']
        )
        results.append({
            'original': item,
            'simplified': result
        })
    return results

In [11]:
print("Testing content simplification functions:")

# Test case 1: Beginner level biology
result1 = simplify_content(
    level="beginner",
    subject="biology", 
    content="photosynthesis process in plants"
)

if result1:
    print(f"\n--- Biology Example ---")
    print(f"Level: beginner")
    print(f"Subject: biology")
    print(f"Content: photosynthesis process in plants")
    print(f"Simplified: {result1}")
    print("-" * 50)

# Test case 2: Advanced level computer science
result2 = simplify_content(
    level="advanced",
    subject="computer science",
    content="machine learning algorithms and neural networks"
)

if result2:
    print(f"\n--- Computer Science Example ---")
    print(f"Level: advanced")
    print(f"Subject: computer science")
    print(f"Content: machine learning algorithms and neural networks")
    print(f"Simplified: {result2}")
    print("-" * 50)

# Test batch processing
print("\n--- Batch Processing Test ---")
batch_content = [
    {
        "level": "intermediate",
        "subject": "physics",
        "content": "Newton's laws of motion"
    },
    {
        "level": "beginner", 
        "subject": "chemistry",
        "content": "atomic structure and electron configuration"
    }
]

batch_results = batch_simplify_content(batch_content)
for i, result in enumerate(batch_results):
    print(f"\nBatch Item {i+1}:")
    print(f"Original: {result['original']}")
    print(f"Simplified: {result['simplified']}")
    print("-" * 30)

Testing prompt template with different variable combinations:
✓ Template loaded successfully
Template preview: You are a Course Content Simplification Agent. Your task is to analyze academic content and adapt it...
Original template length: 1093
Template preview: You are a Course Content Simplification Agent. Your task is to analyze academic content and adapt it to different learning levels.

INPUTS:
- Original academic content: {content}
- Target learning lev...
Variables to fill: {'level': 'beginner', 'subject': 'biology', 'content': 'photosynthesis process in plants'}
Filled prompt length: 1111
Filled prompt preview: You are a Course Content Simplification Agent. Your task is to analyze academic content and adapt it to different learning levels.

INPUTS:
- Original academic content: photosynthesis process in plant...
About to call generate_text with prompt length: 1111

--- Test Case ---
Level: beginner
Subject: biology
Content: photosynthesis process in plants

Response: 
**Complex

In [12]:
try:
    # Close any persistent connections
    model_inference.close_persistent_connection()
    print("Connections closed successfully")
except:
    print("No persistent connections to close")

print("\nNotebook execution completed!")

Connections closed successfully

Notebook execution completed!
